Using Gemini LLM to Match Kaggle/ESPN API School Names to NCES/DoE School Names

In [ ]:
import pandas as pd
import time
import os
from datetime import datetime

import sys
sys.path.append('..')
from utils.llm_utils import call_gemini_api

DELAY = 5 # seconds

# File paths
CFB_DATA_PATH = "../../college_football_data/cfb_box-scores_2002-2024_20251006_193644.csv"
NCES_DATA_PATH = "../../postsecondary_school_locations/postsecondary_school_locations_2020-21_20251006_194802.csv"
MATCHED_SCHOOLS_CSV = "../../intermediate_files/llm_matched_schools.csv"

Read files

In [15]:
nces_df = pd.read_csv(NCES_DATA_PATH)
nces_schools = nces_df['NAME'].tolist()
cfb_df = pd.read_csv(CFB_DATA_PATH)

cfb_schools = set(cfb_df['away'].dropna()) | set(cfb_df['home'].dropna())
cfb_schools = sorted(list(cfb_schools))

print(f"Unique school match count: {len(cfb_schools)}")


Unique school match count: 252


Get existing matches (for partial runs)

In [16]:
if os.path.exists(MATCHED_SCHOOLS_CSV):
    try:
        df = pd.read_csv(MATCHED_SCHOOLS_CSV)
        if len(df) > 0 and 'cfb_school' in df.columns and 'nces_name' in df.columns:
            existing_matches = dict(zip(df['cfb_school'], df['nces_name']))
    except Exception as e:
        existing_matches = {}
else:
    existing_matches = {}

Get list of schools to process

In [17]:
missing_schools = [school for school in cfb_schools if school not in existing_matches]
schools_to_process = [school for school, match in existing_matches.items()
                         if match in ["NO_MATCH", "API_ERROR"]]

if not schools_to_process:
    schools_to_process = [school for school in cfb_schools if school not in existing_matches]


LLM Prompt

In [18]:
def create_matching_prompt(cfb_school, nces_schools_list):
    sample_nces = nces_schools_list[:100]

    prompt = f"""You are a school name matching expert. I need you to find the best matching NCES school name or Department of Education school name for a college football team.

College Football Team: "{cfb_school}"

Here are some examples of NCES school names from the National Center for Education Statistics database:
{chr(10).join(f"- {school}" for school in sample_nces[:20])}

1. Look for the closest match between "{cfb_school}" and NCES school names
2. Consider common variations like:
   - "University" vs "College"
   - Abbreviations (State, A&M, etc.)
   - Location names
   - Ampersands (&) vs "and"
3. If you find a good match, return ONLY the exact NCES school name
4. If no good match exists, return "NO_MATCH"

Return your answer as a single line with just the NCES school name or "NO_MATCH".
Do not include any explanation or additional text."""

    return prompt

Gemini API Wrapping Function

In [19]:
def save_matches(matches_dict):
    df = pd.DataFrame(list(matches_dict.items()), columns=['cfb_school', 'nces_name'])
    # TODO: update the timestamp per row
    df['processed_date'] = datetime.now().isoformat()
    df.to_csv(MATCHED_SCHOOLS_CSV, index=False)
    print("Saved matches")


for i, cfb_school in enumerate(schools_to_process):
    print("Processing school " + cfb_school )

    prompt = create_matching_prompt(cfb_school, nces_schools)
    nces_match, _ = call_gemini_api(prompt)

    existing_matches[cfb_school] = nces_match

    print("Match: " + nces_match)
    print()

    save_matches(existing_matches)
    time.sleep(DELAY)

save_matches(existing_matches)

Processing school Georgia Southern
Match: Georgia Southern University

Saved matches
Processing school Georgia State
Match: Georgia State University

Saved matches
Processing school Georgia Tech
Match: Georgia Institute of Technology-Main Campus

Saved matches
Processing school Grambling State
Match: Grambling State University

Saved matches
Processing school Hampton
Match: Hampton University

Saved matches
Processing school Hawaii
Match: University of Hawaii at Manoa

Saved matches
Processing school Hofstra
Match: Hofstra University

Saved matches
Processing school Holy Cross
Match: College of the Holy Cross

Saved matches
Processing school Houston
Match: University of Houston

Saved matches
Processing school Houston Baptist
Match: Houston Baptist University

Saved matches
Processing school Houston Christian
Match: Houston Christian University

Saved matches
Processing school Howard
Match: Howard University

Saved matches
Processing school Idaho
Match: University of Idaho

Saved match